In [ ]:
from efficientnet_pytorch import EfficientNet

model = EfficientNet.from_pretrained("efficientnet-b3")

In [ ]:
def convert(name):
    if name == "weight":
        return "W"
    if name == "bias":
        return "b"
    return name

kwargs = {}
for name, param in model.named_parameters():
    npz_name = "/".join([convert(n) for n in name.split(".")])
    kwargs[npz_name] = param.detach().numpy()
    if "_se_expand/W" in npz_name or "_se_reduce/W" in npz_name:
        kwargs[npz_name] = kwargs[npz_name][:, :, 0, 0]

In [ ]:
import chainer
import melanoma

weights = np.load("./efficientnetb3.npz")

def load_weights(model, name=""):
    for child in model.children():
        if isinstance(child, chainer.Chain):
            load_weights(child, f"{name}/{child.name}")
        elif isinstance(child, chainer.ChainList):
            for c in child:
                load_weights(c, f"{name}/{child.name}/{c.name}")
        elif isinstance(child, chainer.Link):
            for n, p in child.namedparams():
                param_name = f"{name}/{child.name}{n}"[1:]
                if param_name not in weights:
                    print(f"{param_name} does not exist")
                    continue
                if p.data is None and param_name in weights:
                    print(f"initialize parameter : {param_name}")
                    p.initialize(weights[param_name].shape)
                if p.data.shape != weights[param_name].shape:
                    print(f"shape mismatch : {param_name}. Skip")
                    continue
                p.data[...] = weights[param_name]
                print(f"Load weight {param_name}")
                
                
chainer_model = melanoma.models.EfficientNet(global_params=melanoma.models.EfficientNetB3)
load_weights(chainer_model)                

In [ ]:
chainer.serializers.save_npz("efficientnetb3_chainer.npz", chainer_model)